In [1]:
%cd ..

/Users/chanwutk/Documents/optimized-ingestion


In [2]:
import json
import os
import pickle

from optimized_ingestion.camera_config import camera_config
from optimized_ingestion.payload import Payload
from optimized_ingestion.pipeline import Pipeline
from optimized_ingestion.stages.decode_frame.parallel_decode_frame import ParallelDecodeFrame
from optimized_ingestion.stages.decode_frame.decode_frame import DecodeFrame
from optimized_ingestion.stages.detection_2d.yolo_detection import YoloDetection
from optimized_ingestion.stages.detection_3d.from_2d_and_road import From2DAndRoad as FromD2DAndRoad
from optimized_ingestion.stages.filter_car_facing_sideway import FilterCarFacingSideway
from optimized_ingestion.stages.detection_estimation import DetectionEstimation
from optimized_ingestion.stages.tracking_2d.strongsort import StrongSORT
from optimized_ingestion.stages.tracking_2d.tracking_2d import Tracking2D, Tracking2DResult
from optimized_ingestion.stages.tracking_3d.from_2d_and_road import From2DAndRoad
from optimized_ingestion.stages.tracking_3d.tracking_3d import Tracking3DResult
from optimized_ingestion.stages.segment_trajectory import SegmentTrajectory
from optimized_ingestion.video import Video
from optimized_ingestion.video_skipped import VideoSkipped

/Users/chanwutk/.installs/mambaforge/envs/apperception/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
YOLOv5 🚀 2023-1-11 Python-3.10.8 torch-1.13.1 CPU

Using cache found in /Users/chanwutk/Documents/optimized-ingestion/weights/ultralytics_yolov5_master


Using cpu


YOLOv5 🚀 2023-1-11 Python-3.10.8 torch-1.13.1 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [3]:
# from optimized_ingestion.cache import disable_cache
# disable_cache()

In [4]:
BOSTON_VIDEOS = [
#     "scene-0757-CAM_FRONT",
    # "scene-0103-CAM_FRONT",
    # "scene-0553-CAM_FRONT",
    # "scene-0665-CAM_FRONT",
#     "scene-0655-CAM_FRONT_RIGHT",
    "scene-0655-CAM_BACK_RIGHT",
]

NUSCENES_PROCESSED_DATA = "NUSCENES_PROCESSED_DATA"

In [5]:
import torch

In [6]:
class DataclassJSONEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, Tracking3DResult):
            return {
                "frame_idx": o.frame_idx,
                "detection_id": o.detection_id,
                "object_id": o.object_id,
                "point_from_camera": o.point_from_camera,
                "point": o.point.tolist(),
                "bbox_left": o.bbox_left,
                "bbox_top": o.bbox_top,
                "bbox_w": o.bbox_w,
                "bbox_h": o.bbox_h,
                "object_type": o.object_type,
                "timestamp": str(o.timestamp),
            }
        if isinstance(o, Tracking2DResult):
            return {
                "detection_id": o.detection_id,
                "frame_idx": o.frame_idx,
                "object_id": o.object_id,
                "bbox_left": o.bbox_left,
                "bbox_top": o.bbox_top,
                "bbox_w": o.bbox_w,
                "bbox_h": o.bbox_h,
                "object_type": o.object_type,
                "confidence": o.confidence
            }
        if isinstance(o, torch.Tensor):
            return o.tolist()
        return super().default(o)

In [7]:
print(NUSCENES_PROCESSED_DATA in os.environ)
print(os.environ['NUSCENES_PROCESSED_DATA'])

True
/Users/chanwutk/Documents/data/nuscenes-mini


In [8]:
if NUSCENES_PROCESSED_DATA in os.environ:
    DATA_DIR = os.environ[NUSCENES_PROCESSED_DATA]
else:
    DATA_DIR = "/work/apperception/data/nuScenes/full-dataset-v1.0/Mini"
with open(os.path.join(DATA_DIR, "videos/boston-seaport", "frames-skip.pickle"), "rb") as f:
    videos = pickle.load(f)

In [ ]:
cache_pipeline = Pipeline()
cache_pipeline.add_filter(ParallelDecodeFrame())
cache_pipeline.add_filter(YoloDetection())

for name, video in videos.items():
    frames = VideoSkipped(
        os.path.join(DATA_DIR, "videos/boston-seaport", video["filename"]),
        [camera_config(*f, 0) for f in video["frames"]],
    )

    cache_pipeline.run(Payload(frames))

YOLOv5 🚀 2023-1-11 Python-3.10.8 torch-1.13.1 CPU

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /Users/chanwutk/Documents/weights/master.zip
YOLOv5 🚀 2023-1-11 Python-3.10.8 torch-1.13.1 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


Stage:  DecodeFrame.ParallelDecodeFrame


100%|███████████████████████████████████████████████████████████████████████████████████| 16/16 [00:18<00:00,  1.18s/it]


None
389
  filtered frames: 59.383033419023135%
K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.
K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.K..K.KK.K.KK.K.KK.K.KK.K.KK.
K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K..K.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.
K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.K..K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK
Stage:  Detection2D.YoloDetection


100%|█████████████████████████████████████████████████████████████████████████████████| 389/389 [00:27<00:00, 14.32it/s]


...

100%|█████████████████████████████████████████████████████████████████████████████████| 399/399 [00:24<00:00, 15.97it/s]


None
399
  filtered frames: 59.147869674185465%
K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.
K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.K..K.KK.K.K..K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.
K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK...KK.K.KK.K.KK.K.KK.K.KK.
K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK...KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK
Stage:  DecodeFrame.ParallelDecodeFrame


100%|███████████████████████████████████████████████████████████████████████████████████| 16/16 [00:12<00:00,  1.25it/s]


None
399
  filtered frames: 58.89724310776943%
K.KK.K.KK.K..K.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.
K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K..K.K.KK.K.KK.K.KK.K.KK.K.KK.
K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K..K.
K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.K..K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K.KK.K..K.K.KK.K.KK.K.KK.K.KK.K.KK
Stage:  Detection2D.YoloDetection


 53%|██████████████████████████████████████████▋                                      | 210/399 [00:12<00:10, 17.28it/s]


In [ ]:
pipeline = Pipeline()
pipeline.add_filter(filter=ParallelDecodeFrame())
pipeline.add_filter(filter=YoloDetection())

pipeline.add_filter(filter=FromD2DAndRoad())
pipeline.add_filter(filter=DetectionEstimation())
pipeline.add_filter(filter=StrongSORT())

pipeline.add_filter(filter=From2DAndRoad())
pipeline.add_filter(filter=SegmentTrajectory())

metadata = {}
outputs_with_estimation = []
for name, video in videos.items():
    if not name.endswith('CAM_FRONT'):
        continue
    print(name, '--------------------------------------------------------------------------------')
    frames = VideoSkipped(
        os.path.join(DATA_DIR, "videos/boston-seaport", video["filename"]),
        [camera_config(*f, 0) for f in video["frames"]],
    )

    output = pipeline.run(Payload(frames))
    outputs_with_estimation.append(output)
    metadata[name] = SegmentTrajectory.get(output)

In [ ]:
with open(f"./outputs/trackings-with-estimation.json", "w") as f:
    json.dump(metadata, f, cls=DataclassJSONEncoder)

In [ ]:
pipeline = Pipeline()
pipeline.add_filter(filter=ParallelDecodeFrame())
pipeline.add_filter(filter=YoloDetection())

# pipeline.add_filter(filter=DetectionEstimation())
pipeline.add_filter(filter=StrongSORT())

pipeline.add_filter(filter=From2DAndRoad())

if NUSCENES_PROCESSED_DATA in os.environ:
    DATA_DIR = os.environ[NUSCENES_PROCESSED_DATA]
else:
    DATA_DIR = "/work/apperception/data/nuScenes/full-dataset-v1.0/Mini"
with open(os.path.join(DATA_DIR, "videos/boston-seaport", "frames.pickle"), "rb") as f:
    videos = pickle.load(f)

metadata = {}
outputs_without_estimation = []
for name, video in videos.items():
#     if name not in BOSTON_VIDEOS:
#         continue
    if not name.endswith('CAM_FRONT'):
        continue
    print(name, '--------------------------------------------------------------------------------')
    frames = Video(
        os.path.join(DATA_DIR, "videos/boston-seaport", video["filename"]),
        [camera_config(*f, 0) for f in video["frames"]],
        video["start"],
    )

    output = pipeline.run(Payload(frames))
    outputs_without_estimation.append(output)
    metadata[name] = From2DAndRoad.get(output)

In [ ]:
with open(f"./outputs/trackings-without-estimation.json", "w") as f:
    json.dump(metadata, f, cls=DataclassJSONEncoder)

In [ ]:
res = Tracking2D.get(outputs_without_estimation[1])

In [ ]:
res